# July 12, 13, 16, 17

In [3]:


import torch
import torch.nn.functional as F
import time

dist_matrix = torch.rand(1000, 120)
print(dist_matrix.shape)
num_nearest_neighbors = 5

start = time.time() 
ind = torch.topk(dist_matrix, num_nearest_neighbors, largest=False).indices
end = time.time() 

print(f"Time taken to run the function: {end - start} seconds")
print(ind.shape)


torch.Size([1000, 120])
Time taken to run the function: 0.0024809837341308594 seconds
torch.Size([1000, 5])


In [4]:
import torch
import numpy as np

B, C, N = 32, 12, 40
n_samples = 5
k_neig = 5

ex = torch.rand(B, C, N)
print("Ex shape: ", ex.shape)

M = ex[0]/torch.norm(ex[0], dim=0)  # Pick first item in patch and normalize the rows
print("M shape: ", M.shape)

rand_idx = np.random.choice(M.shape[1], n_samples, replace=False) 
print(rand_idx)

M_prime = M[:, rand_idx]
print("M prime shape: ", M_prime.shape)

sim = M.T@M_prime
print("Sim shape: ", sim.shape)


# Make similarities between samples equal to -inf, so they are not considered when using topk
sim[rand_idx, np.arange(n_samples)]  =  -np.inf  
# print(sim[:rand_idx[0]])
# print(sim[:rand_idx[0]].shape)


# Only get k-1 neigbors (we'll add the selves later as the first nearest neigbor) 
_, indxs = torch.topk(sim, k_neig - 1, largest=True) 
b = [rand_idx[x] for x in indxs]
neig1 = M[:, b]
print("Neig shape: ", neig1.shape)
print("M.unsqueeze(2) shape: ", M.unsqueeze(2).shape)

# Add the selves as the first nearest neigbor
neig2 = torch.concat([M.unsqueeze(2), neig1], dim=2)
print(neig2.shape)


Ex shape:  torch.Size([32, 12, 40])
M shape:  torch.Size([12, 40])
[35  5 11 39 18]
M prime shape:  torch.Size([12, 5])
Sim shape:  torch.Size([40, 5])
Neig shape:  torch.Size([12, 40, 4])
M.unsqueeze(2) shape:  torch.Size([12, 40, 1])
torch.Size([12, 40, 5])


/var/folders/s7/md5b66vn4l39df_j063nqlp00000gn/T/ipykernel_95647/2407026106.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  neig1 = M[:, b]


In [5]:
print(rand_idx)
c = rand_idx[0]
print(c)
torch.concat([torch.arange(40).unsqueeze(1), sim], dim=1)

[35  5 11 39 18]
35


tensor([[ 0.0000,  0.6918,  0.6979,  0.8874,  0.7272,  0.7101],
        [ 1.0000,  0.7906,  0.7905,  0.7762,  0.8334,  0.7995],
        [ 2.0000,  0.6744,  0.6935,  0.8854,  0.5431,  0.7086],
        [ 3.0000,  0.8333,  0.7337,  0.7377,  0.6393,  0.8565],
        [ 4.0000,  0.7755,  0.7808,  0.7749,  0.8573,  0.8554],
        [ 5.0000,  0.7924,  1.0000,  0.7089,  0.6049,  0.7891],
        [ 6.0000,  0.8282,  0.7659,  0.6740,  0.5203,  0.6992],
        [ 7.0000,  0.8019,  0.7339,  0.6530,  0.7254,  0.7017],
        [ 8.0000,  0.6802,  0.7508,  0.7341,  0.8118,  0.7243],
        [ 9.0000,  0.8102,  0.9141,  0.8046,  0.5730,  0.8086],
        [10.0000,  0.8104,  0.8393,  0.8194,  0.8174,  0.8018],
        [11.0000,  0.8184,  0.7089,  1.0000,  0.6787,  0.7481],
        [12.0000,  0.7721,  0.7163,  0.6784,  0.6350,  0.8817],
        [13.0000,  0.7438,  0.6295,  0.6880,  0.6045,  0.6814],
        [14.0000,  0.7740,  0.6331,  0.7568,  0.6083,  0.7601],
        [15.0000,  0.7548,  0.8500,  0.7

In [6]:
M[:, c]

tensor([0.1453, 0.4398, 0.3740, 0.3932, 0.1829, 0.4559, 0.2376, 0.0693, 0.0573,
        0.2749, 0.3261, 0.0567])

In [7]:
neig1[:, c, :].T

tensor([[0.1453, 0.4398, 0.3740, 0.3932, 0.1829, 0.4559, 0.2376, 0.0693, 0.0573,
         0.2749, 0.3261, 0.0567],
        [0.3231, 0.2965, 0.0038, 0.3717, 0.1529, 0.4523, 0.3680, 0.2733, 0.1466,
         0.0133, 0.3904, 0.2395],
        [0.3486, 0.2537, 0.2246, 0.4832, 0.1932, 0.0271, 0.0957, 0.1651, 0.1602,
         0.4067, 0.4903, 0.1560],
        [0.2857, 0.1100, 0.4676, 0.1616, 0.4580, 0.1462, 0.4051, 0.1387, 0.0167,
         0.2577, 0.3864, 0.1761]])

In [8]:
neig2[:, c, :].T

tensor([[0.1453, 0.4398, 0.3740, 0.3932, 0.1829, 0.4559, 0.2376, 0.0693, 0.0573,
         0.2749, 0.3261, 0.0567],
        [0.1453, 0.4398, 0.3740, 0.3932, 0.1829, 0.4559, 0.2376, 0.0693, 0.0573,
         0.2749, 0.3261, 0.0567],
        [0.3231, 0.2965, 0.0038, 0.3717, 0.1529, 0.4523, 0.3680, 0.2733, 0.1466,
         0.0133, 0.3904, 0.2395],
        [0.3486, 0.2537, 0.2246, 0.4832, 0.1932, 0.0271, 0.0957, 0.1651, 0.1602,
         0.4067, 0.4903, 0.1560],
        [0.2857, 0.1100, 0.4676, 0.1616, 0.4580, 0.1462, 0.4051, 0.1387, 0.0167,
         0.2577, 0.3864, 0.1761]])

# I. Distance/Similarity Matrix check

### Distance matrix sanity check for all samples

In [9]:
def calculate_distance_matrix(matrix): 
   '''Calculating the distance matrix of the input matrix'''
   norm_squared = torch.sum(matrix ** 2, dim=1, keepdim=True)
   dot_product = torch.bmm(matrix.transpose(2, 1), matrix)
   dist_matrix = norm_squared + norm_squared.transpose(2, 1) - 2 * dot_product
   return torch.sqrt(dist_matrix) # May need to remove torch.sqrt - do not need that computation


### Distance Matrix sanity check for N Sample 

In [10]:
def calculate_distance_matrix_N(m1, m2):
   '''Calculate the distance matrix between two input matrices'''
   # to make it simple -> normalize the rows, normalize across c (channels), 
   norm_squared_1 = torch.sum(m1 ** 2, dim=1, keepdim=True)
   norm_squared_2 = torch.sum(m2 ** 2, dim=1, keepdim=True).transpose(2, 1)
   dot_product = torch.bmm(m1.transpose(2, 1), m2)

   norm_squared_1 = norm_squared_1.permute(0, 2, 1)  
   norm_squared_2 = norm_squared_2.permute(0, 2, 1)  

   dist_matrix = norm_squared_1 + norm_squared_2 - 2 * dot_product
   return torch.sqrt(dist_matrix)

def calculate_distance_matrix_N_v2(matrix, matrix_sample):
   '''Calculate distance matrix between two input matrices''' 
   norm_squared = torch.sum(matrix ** 2, dim=1, keepdim=True).permute(0, 2, 1)
   norm_squared_sample = torch.sum(matrix_sample ** 2, dim=1, keepdim=True).transpose(2, 1).permute(0, 2, 1)
   dot_product = torch.bmm(matrix.transpose(2, 1), matrix_sample)
   dist_matrix = norm_squared + norm_squared_sample - 2 * dot_product
   return torch.sqrt(dist_matrix)
   
   


In [11]:
def dist_matrix(m1, m2): 
   # Calculate the distance matrix using for-loops 
   dist_matrix = torch.zeros(m1.shape[0], m1.shape[2], m2.shape[2])
   
   for i in range(m1.shape[0]): 
      for j in range(m1.shape[2]): 
            for k in range(m2.shape[2]): 
               dist_matrix[i, j, k] = torch.norm(m1[i, :, j] -  m2[i, :, k])
   return dist_matrix

In [12]:
m1 = torch.rand(32, 12, 40)
m2 = torch.rand(32, 12, 5)

a = calculate_distance_matrix_N(m1, m2)
b = dist_matrix(m1, m2)
c = calculate_distance_matrix_N_v2(m1, m2)
print(a.shape)
print(b.shape)

torch.Size([32, 40, 5])
torch.Size([32, 40, 5])


In [13]:
print(torch.allclose(a, b))
print(torch.allclose(a, c))
print(torch.allclose(b, c))



True
True
True


### Similarity Matrix sanity check for N Sample

In [14]:
def sim_a(matrix): 
   '''Calculate the similarity matrix of the input matrix'''
   normalized_matrix = F.normalize(matrix, p=2, dim=1) # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
   dot_product = torch.bmm(normalized_matrix.transpose(2, 1), normalized_matrix)
   similarity_matrix = dot_product 
   return similarity_matrix


In [15]:
def sim_b(m):
    # Normalize the input matrix
    normalized_matrix = F.normalize(m, p=2, dim=1)
    
    # Initialize the similarity matrix with zeros
    similarity_matrix = torch.zeros(m.size(0), m.size(2), m.size(2))
    
    # Calculate the similarity (dot product) between each pair of vectors
    for i in range(m.shape[0]): 
      for j in range(m.shape[2]): 
            for k in range(m.shape[2]): 
               similarity_matrix[i, j, k] = torch.dot(normalized_matrix[i, :, j],  normalized_matrix[i, :, k])
    
    return similarity_matrix


In [16]:
m = torch.rand(32, 12, 40)
sim1 = sim_a(m)
sim2 = sim_b(m)

print(sim1.shape)
print(sim2.shape)


print(sim1[0, 0, :3])
print(sim2[0, 0, :3]) 

print(torch.allclose(sim1, sim2))



torch.Size([32, 40, 40])
torch.Size([32, 40, 40])
tensor([1.0000, 0.6578, 0.7064])
tensor([1.0000, 0.6578, 0.7064])
True


In [17]:
## For N Samples

def sim_c(m1, m2): 
   normalized_matrix_1 = F.normalize(m1, p=2, dim=1) # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
   normalized_matrix_2 = F.normalize(m2, p=2, dim=1)
   
   dot_product = torch.bmm(normalized_matrix_1.transpose(2, 1), normalized_matrix_2)
            
   similarity_matrix = dot_product 
   return similarity_matrix

def calculate_similarity_matrix_N(matrix, matrix_sample): 
   normalized_matrix = F.normalize(matrix, p=2, dim=1) # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
   normalized_matrix_sample = F.normalize(matrix_sample, p=2, dim=1)
   similarity_matrix = dot_product = torch.bmm(normalized_matrix.transpose(2, 1), normalized_matrix_sample)
   return similarity_matrix

def sim_d(m1, m2): 
   
   norm1 = F.normalize(m1, p=2, dim=1)
   norm2 = F.normalize(m2, p=2, dim=1)
       
    # Initialize the similarity matrix with zeros
   similarity_matrix = torch.zeros(m.size(0), m1.size(2), m2.size(2))

   # Calculate the similarity (dot product) between each pair of vectors
   for i in range(m1.shape[0]): 
      for j in range(m1.shape[2]): 
            for k in range(m2.shape[2]): 
               similarity_matrix[i, j, k] = torch.dot(norm1[i, :, j],  norm2[i, :, k])

   return similarity_matrix

In [18]:
m1 = torch.rand(32, 12, 40)
m2 = torch.rand(32, 12, 5)

sim3 = sim_c(m1, m2)
sim4 = sim_d(m1, m2)
sim5 = calculate_similarity_matrix_N(m1, m2)

print()

print(torch.allclose(sim3, sim4))
print(torch.allclose(sim3, sim5))
print(torch.allclose(sim4, sim5))


True
True
True


# II. Prime comparisons

### Sanity check for topk for distance prime

In [19]:
def prime(matrix, dist_matrix, num_nearest_neighbors): 
   '''Returns the convolution matrix of the NNT object using for-loops and vectorization'''
   ### Note: For-Loop Implementation and Vectorization Implementation are the same
   
   ##For-Loop Implementation
   
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      
       concat_list = [] 
       for j in range(matrix.shape[2]): 
           # Get the indices of the nearest neighbors
           indices = torch.topk(dist_matrix[i, j, :], num_nearest_neighbors, largest=False).indices
            
           # Get the nearest neighbors
           nearest_neighbors = matrix[i, :, indices]
            
           # Concatenate the nearest neighbors
           concat_list.append(nearest_neighbors)
      
       # Concatenate the tensor list to create the convolution matrix 
       concat = torch.cat(concat_list, dim=1)
       stack_list.append(concat)
   prime = torch.stack(stack_list, dim= 0)
   
   
   
   return prime

def prime_vmap_2d(matrix, magnitude_matrix, num_nearest_neighbors, largest=False): 
   # Vectorization / Vmap Implementation for Nearest Neighbor Tensor 2D
   batched_process = torch.vmap(process_batch, in_dims=(0, 0, None), out_dims=0)
   prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=largest)
   return prime 

def process_batch(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=False):
      '''Process the batch of matrices''' 
      ind = torch.topk(magnitude_matrix, num_nearest_neighbors, largest=largest).indices
      neigh = matrix[:, ind]
      if flatten: 
         reshape = torch.flatten(neigh, start_dim=1)
         return reshape
      else: 
         return neigh
      


In [20]:
m = torch.rand(32, 12, 40)
dist = calculate_distance_matrix(m)

p1 = prime(m, dist, 5)
p2 = prime_vmap_2d(m, dist, 5)

In [21]:
print(p1[0, 0, :3])
print(p2[0, 0, :3])

print(torch.allclose(p1, p2))

tensor([0.3093, 0.3509, 0.1961])
tensor([0.3093, 0.3509, 0.1961])
True


### Sanity check for similarity prime

In [22]:
def sim_prime(matrix, sim_mat, num_nearest_neighbors): 
   '''Returns the convolution matrix of the NNT object using for-loops and vectorization'''
   ### Note: For-Loop Implementation and Vectorization Implementation are the same
   
   ##For-Loop Implementation
   
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      
       concat_list = [] 
       for j in range(matrix.shape[2]): 
           # Get the indices of the nearest neighbors
           indices = torch.topk(sim_mat[i, j, :], num_nearest_neighbors, largest=True).indices
            
           # Get the nearest neighbors
           nearest_neighbors = matrix[i, :, indices]
            
           # Concatenate the nearest neighbors
           concat_list.append(nearest_neighbors)
      
       # Concatenate the tensor list to create the convolution matrix 
       concat = torch.cat(concat_list, dim=1)
       stack_list.append(concat)
   prime = torch.stack(stack_list, dim= 0)
   
   
   return prime

def sim_prime_vmap_2d(matrix, magnitude_matrix, num_nearest_neighbors, largest=True): 
   # Vectorization / Vmap Implementation for Nearest Neighbor Tensor 2D
   batched_process = torch.vmap(process_batch, in_dims=(0, 0, None), out_dims=0)
   prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=largest)
   return prime 

def sim_process_batch(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=False):
      '''Process the batch of matrices''' 
      ind = torch.topk(magnitude_matrix, num_nearest_neighbors, largest=largest).indices
      neigh = matrix[:, ind]
      if flatten: 
         reshape = torch.flatten(neigh, start_dim=1)
         return reshape
      else: 
         return neigh

In [23]:
m = torch.rand(32, 12, 40)
sim = sim_a(m)

p1 = sim_prime(m, sim, 5)
p2 = sim_prime_vmap_2d(m, sim, 5)

In [24]:
print(p1[0, 0:3, :3])
print(p2[0, 0:3, :3])

print(torch.allclose(p1, p2))

print(p1.shape)

tensor([[0.8638, 0.8472, 0.8591],
        [0.4595, 0.5199, 0.7578],
        [0.7437, 0.4739, 0.5776]])
tensor([[0.8638, 0.8472, 0.8591],
        [0.4595, 0.5199, 0.7578],
        [0.7437, 0.4739, 0.5776]])
True
torch.Size([32, 12, 200])


### Distance prime for N sample

In [35]:
# Og shape = [32, 12, 40] 
# prime1 shape = [32, 40, 5]
# prime shape = [32, 12, 200]

indices_list1 = [] 
indices_list2 = [] 

def prime_brute_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, largest = False):         
   stack_list = [] 

   for i in range(matrix.shape[0]): 
      
      concat_list = []
      
      for j in range(matrix.shape[2]): 
            
         if j in rand_idx: 
            indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors, largest=largest).indices
            indices_list = [rand_idx[i] for i in indices]
            nearest_neighbors = matrix[i, :, indices_list]
         else: 
            indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors - 1, largest=largest).indices
            indices_list = [j] + [rand_idx[i] for i in indices]
            nearest_neighbors = matrix[i, :, indices_list]
            
         indices_list1.append(indices_list)
            
         concat_list.append(nearest_neighbors)
      
      stacked_neighbors = torch.cat(concat_list, dim=1)
      stack_list.append(stacked_neighbors)
   

   prime = torch.stack(stack_list, dim= 0)
   return prime

def prime_brute_N_v2(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx,largest=False): 
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      concat_list = [] 
      
      for j in range(matrix.shape[2]): 
         indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors - 1, largest=largest).indices
         indices_list = [j] + [rand_idx[i] for i in indices] 
         nearest_neighbors = matrix[i, :, indices_list]
         concat_list.append(nearest_neighbors)
   
         indices_list2.append(indices_list)
         
      stacked_neighbors = torch.cat(concat_list, dim=1)
      stack_list.append(stacked_neighbors)
   
   prime = torch.stack(stack_list, dim=0)
   return prime

rand_idx = [3, 8, 14, 28, 39]
nnn = 3

m1 = torch.rand(32, 12, 40)
m2 = m1[:, :, rand_idx]
dist = calculate_distance_matrix_N(m1, m2)

p1 = prime_brute_N(m1, dist, nnn, rand_idx, False)

dist[:, rand_idx, np.arange(len(rand_idx))] = np.inf

p2 = prime_brute_N_v2(m1, dist, nnn, rand_idx, False) 

print(torch.equal(p1, p2))

print(p1[0, 0, :20])
print(p2[0, 0, :20])


differing_indices = [i for i, (a, b) in enumerate(zip(indices_list1, indices_list2)) if a != b]

# Print the indices
print(differing_indices)
print(indices_list1[3])
print(indices_list2[3])


False
tensor([0.6720, 0.3938, 0.1533, 0.2187, 0.1533, 0.4430, 0.9430, 0.3938, 0.1533,
        0.0167, 0.0978, 0.4430, 0.6502, 0.4430, 0.0167, 0.5355, 0.3938, 0.4430,
        0.3531, 0.4430])
tensor([0.6720, 0.3938, 0.1533, 0.2187, 0.1533, 0.4430, 0.9430, 0.3938, 0.1533,
        0.0167, 0.0978, 0.4430, 0.6502, 0.4430, 0.0167, 0.5355, 0.3938, 0.4430,
        0.3531, 0.4430])
[83, 119, 239, 279, 283, 319, 328, 359, 374, 388, 439, 508, 523, 639, 683, 788, 799, 814, 839, 854, 919, 934, 963, 968, 1014, 1039, 1079, 1083, 1088, 1108, 1119, 1248]
[3, 39, 14]
[3, 39, 14]


In [41]:
# differing_indices = [i for i, (a, b) in enumerate(zip(indices_list1, indices_list2)) if a != b]

# # Print the indices
# print(indices_list1[8])
# print(indices_list2[8])

# print(indices_list1[9])

print(np.array(indices_list2) - np.array(indices_list1))


[[0 0 0]
 [0 0 0]
 [0 0 0]
 ...
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [32]:
import torch

def prime_vmap_2d_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, largest=False):
    
    batched_process = torch.vmap(process_batch_N, in_dims=(0, 0, None, None), out_dims=0)
    prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, flatten=True, largest=largest)
    return prime

def process_batch_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, flatten=True, largest=False):
    topk_ind = torch.topk(magnitude_matrix, num_nearest_neighbors - 1, largest=largest).indices
    mapped_tensor = rand_idx[topk_ind]
    index_tensor = torch.arange(0, matrix.shape[1]).unsqueeze(1) # shape [40, 1]
    final_tensor = torch.cat([index_tensor, mapped_tensor], dim=1)
    print(final_tensor)
    
         
    neigh = matrix[:, final_tensor]
    if flatten:
        reshape = torch.flatten(neigh, start_dim=1)
        return reshape
    else:
        return neigh

In [33]:
rand_idx = torch.tensor([3, 8, 14, 28, 39])
rand_idx = [3, 8, 14, 28, 39]


print(m1.shape) 
print(dist.shape)
p3 = prime_vmap_2d_N(m1, dist, nnn, torch.tensor(rand_idx), False)
print(p3.shape)

torch.Size([32, 12, 40])
torch.Size([32, 40, 5])
BatchedTensor(lvl=1, bdim=0, value=
    tensor([[[ 0, 28,  3],
             [ 1, 28, 14],
             [ 2, 28,  3],
             ...,
             [37,  8,  3],
             [38,  3, 14],
             [39, 28, 14]],

            [[ 0, 39,  8],
             [ 1,  8, 14],
             [ 2, 14,  8],
             ...,
             [37,  3, 28],
             [38, 14,  8],
             [39, 28,  3]],

            [[ 0,  3, 39],
             [ 1,  3, 39],
             [ 2, 28, 39],
             ...,
             [37, 14,  8],
             [38, 28,  8],
             [39,  8, 28]],

            ...,

            [[ 0,  8, 28],
             [ 1,  3, 28],
             [ 2, 39, 28],
             ...,
             [37, 39, 14],
             [38, 14, 39],
             [39,  3, 14]],

            [[ 0,  3, 39],
             [ 1,  3, 39],
             [ 2,  3, 14],
             ...,
             [37,  3, 14],
             [38,  3, 14],
             [39

In [29]:
print(torch.allclose(p2, p3))


True


In [30]:
import torch

# Example tensor of size [40, 2]
original_tensor = torch.randint(0, 5, (40, 2))

# Mapping list: the index corresponds to the number in the original tensor
mapping_list = [3, 8, 14, 28, 39]

# Convert the mapping list to a tensor
mapping_tensor = torch.tensor(mapping_list)

# Use the original tensor to index the mapping tensor
mapped_tensor = mapping_tensor[original_tensor]

# Create a tensor that adds numbers from 0 to 39
index_tensor = torch.arange(40).unsqueeze(1)  # shape [40, 1]

# Concatenate the index tensor with the mapped tensor
final_tensor = torch.cat((index_tensor, mapped_tensor), dim=1)

print(original_tensor[:3, :])
print(mapped_tensor[:3, :])
print(final_tensor[:3, :])

tensor([[2, 1],
        [4, 3],
        [0, 1]])
tensor([[14,  8],
        [39, 28],
        [ 3,  8]])
tensor([[ 0, 14,  8],
        [ 1, 39, 28],
        [ 2,  3,  8]])


In [31]:
rand_idx = [3, 8, 14, 28, 39]
nnn = 3

m1 = torch.rand(32, 12, 40)
m2 = m1[:, :, rand_idx]
dist = calculate_distance_matrix_N(m1, m2)

dist[:, rand_idx, np.arange(len(rand_idx))] = np.inf
print(dist[0, 0:10, :])
print(dist.shape)

ind = torch.topk(dist, nnn-1, largest=False).indices
print(ind.shape)


tensor([[1.3040, 1.3568, 1.3979, 1.2008, 1.6860],
        [1.3195, 1.2407, 1.0779, 0.9359, 1.3165],
        [1.0957, 1.2363, 1.1946, 1.0849, 1.1731],
        [   inf, 1.6678, 1.3483, 1.0666, 1.4366],
        [1.5869, 2.0976, 1.2488, 1.7155, 1.7232],
        [0.7085, 1.6939, 1.4109, 1.3193, 1.4974],
        [1.4074, 1.3709, 1.3628, 1.5359, 1.6381],
        [1.2147, 1.7285, 1.4156, 1.1246, 1.6689],
        [1.6678,    inf, 1.3386, 1.2787, 1.5047],
        [1.2714, 1.0765, 1.2748, 0.9211, 1.3587]])
torch.Size([32, 40, 5])
torch.Size([32, 40, 2])
